## Imports
Magic commands:
- Reload modules before executing user code.
- Show matplotlib graphs in notebook style.

In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
from fastai.imports import *
from fastai.structured import *

from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display

from sklearn import metrics
from scipy import stats

In [3]:
PATH = "../data/crime-data-in-brazil/"

In [4]:
!ls {PATH}

BO_2007_1.csv  BO_2010_2.csv  BO_2014_1.csv  RDO_column_description.csv
BO_2007_2.csv  BO_2011_1.csv  BO_2014_2.csv  RDO_column_description_eng.csv
BO_2008_1.csv  BO_2011_2.csv  BO_2015.csv    RDO_methodology.txt
BO_2008_2.csv  BO_2012_1.csv  BO_2016.csv    tmp
BO_2009_1.csv  BO_2012_2.csv  RDO_1.csv
BO_2009_2.csv  BO_2013_1.csv  RDO_2.csv
BO_2010_1.csv  BO_2013_2.csv  RDO_3.csv


In [5]:
df_raw = pd.read_csv(f'{PATH}BO_2009_1.csv')

/home/sam/anaconda3/envs/fastai-cpu/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (26,32,33) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
types = {
'NUM_BO': np.int32,
'ANO_BO': np.int16,
'ID_DELEGACIA': np.int32,
'NOME_DEPARTAMENTO': 'object',
'NOME_SECCIONAL': 'object',
'DELEGACIA': 'object',
'NOME_DEPARTAMENTO_CIRC': 'object',
'NOME_SECCIONAL_CIRC': 'object',
'NOME_DELEGACIA_CIRC': 'object',
'ANO': np.int32,
'MES': np.int8,
'DATA_OCORRENCIA_BO': 'object',
'HORA_OCORRENCIA_BO': 'object',
'FLAG_STATUS': 'object',
'RUBRICA': 'object',
'DESDOBRAMENTO': 'object',
'CONDUTA': 'object',
'LATITUDE': 'object',
'LONGITUDE': 'object',
'CIDADE': 'object',
'LOGRADOURO': 'object',
'NUMERO_LOGRADOURO': 'object',
'FLAG_STATUS.1': 'object',
'DESCR_TIPO_PESSOA': 'object',
'CONT_PESSOA': np.float32,
'SEXO_PESSOA': 'object',
'IDADE_PESSOA': 'object',
'COR': 'object',
'DESCR_PROFISSAO': 'object',
'DESCR_GRAU_INSTRUCAO': 'object',
'Unnamed:30': 'object',
'Unnamed:31': 'object',
'Unnamed:32': 'object',
'Unnamed:33': 'object'	
}

In [7]:
df_raw = pd.read_csv(f'{PATH}BO_2009_1.csv', dtype=types, parse_dates=['DATA_OCORRENCIA_BO', 'HORA_OCORRENCIA_BO'])

/home/sam/anaconda3/envs/fastai-cpu/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (32,33) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
int_cols = ['CONT_PESSOA']
df_raw[int_cols] = df_raw[int_cols].fillna(df_raw[int_cols].median())
df_raw[int_cols] = df_raw[int_cols].apply(pd.to_numeric, downcast='integer')

In [9]:
df_raw.dtypes

NUM_BO                             int32
ANO_BO                             int16
ID_DELEGACIA                       int32
NOME_DEPARTAMENTO                 object
NOME_SECCIONAL                    object
DELEGACIA                         object
NOME_DEPARTAMENTO_CIRC            object
NOME_SECCIONAL_CIRC               object
NOME_DELEGACIA_CIRC               object
ANO                                int32
MES                                 int8
DATA_OCORRENCIA_BO        datetime64[ns]
HORA_OCORRENCIA_BO        datetime64[ns]
FLAG_STATUS                       object
RUBRICA                           object
DESDOBRAMENTO                     object
CONDUTA                           object
LATITUDE                          object
LONGITUDE                         object
CIDADE                            object
LOGRADOURO                        object
NUMERO_LOGRADOURO                 object
FLAG_STATUS.1                     object
DESCR_TIPO_PESSOA                 object
CONT_PESSOA     

In [10]:
df_raw.drop(df_raw.columns[df_raw.columns.str.contains('Unnamed',case = False)],axis = 1, inplace = True)

In [11]:
df_raw.describe()

,NUM_BO,ANO_BO,ID_DELEGACIA,ANO,MES,CONT_PESSOA
count,8.017900e+05,801790.000000,801790.000000,801790.0,801790.000000,801790.000000
mean,2.892315e+04,2009.010906,99004.420438,2009.0,3.530321,2.175941
std,1.178970e+05,0.144315,249542.484383,0.0,1.693753,1.624671
min,1.000000e+00,2009.000000,10004.000000,2009.0,1.000000,1.000000
25%,8.090000e+02,2009.000000,10336.000000,2009.0,2.000000,1.000000
50%,1.759000e+03,2009.000000,20218.000000,2009.0,4.000000,2.000000
75%,3.082000e+03,2009.000000,30218.000000,2009.0,5.000000,3.000000
max,1.327690e+06,2017.000000,900833.000000,2009.0,6.000000,48.000000


In [12]:
def display_all(df):
    with pd.option_context("display.max_rows", 1000, "display.max_columns", 1000): 
        display(df)

In [13]:
display_all(df_raw.tail().T)

,801785,801786,801787,801788,801789
NUM_BO,900666,900687,900726,900747,913970
ANO_BO,2009,2009,2009,2009,2011
ID_DELEGACIA,30427,30406,30427,30427,900020
NOME_DEPARTAMENTO,DEMACRO,DEMACRO,DEMACRO,DEMACRO,DIPOL - DEPTO DE INTELIGENCIA
NOME_SECCIONAL,DEL.SEC.CARAPICUIBA,DEL.SEC.TABOÃO DA SERRA,DEL.SEC.CARAPICUIBA,DEL.SEC.CARAPICUIBA,DELEGACIA ELETRONICA
DELEGACIA,01º D.P. CARAPICUIBA,DEL.POL.ITAPECERICA DA SERRA,01º D.P. CARAPICUIBA,01º D.P. CARAPICUIBA,DELEGACIA ELETRONICA
NOME_DEPARTAMENTO_CIRC,DEMACRO,DEMACRO,DEMACRO,DEMACRO,DECAP
NOME_SECCIONAL_CIRC,DEL.SEC.CARAPICUIBA,DEL.SEC.TABOÃO DA SERRA,DEL.SEC.CARAPICUIBA,DEL.SEC.CARAPICUIBA,DEL.SEC.4º NORTE
NOME_DELEGACIA_CIRC,01º D.P. CARAPICUIBA,DEL.POL.ITAPECERICA DA SERRA,02º D.P. CARAPICUIBA,02º D.P. CARAPICUIBA,19º D.P. VILA MARIA
ANO,2009,2009,2009,2009,2009


In [14]:
display_all(df_raw.describe(include='all').T)

,count,unique,top,freq,first,last,mean,std,min,25%,50%,75%,max
NUM_BO,801790,NaN,NaN,NaN,NaN,NaN,28923.2,117897,1,809,1759,3082,1.32769e+06
ANO_BO,801790,NaN,NaN,NaN,NaN,NaN,2009.01,0.144315,2009,2009,2009,2009,2017
ID_DELEGACIA,801790,NaN,NaN,NaN,NaN,NaN,99004.4,249542,10004,10336,20218,30218,900833
NOME_DEPARTAMENTO,801790,19,DECAP,464202,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NOME_SECCIONAL,801790,67,DEL.SEC.3º OESTE,74332,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DELEGACIA,801790,469,DELEGACIA ELETRONICA,69804,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NOME_DEPARTAMENTO_CIRC,801790,2,DECAP,531941,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NOME_SECCIONAL_CIRC,801790,17,DEL.SEC.3º OESTE,89931,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NOME_DELEGACIA_CIRC,801790,182,01º D.P. SE,17109,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ANO,801790,NaN,NaN,NaN,NaN,NaN,2009,0,2009,2009,2009,2009,2009


In [15]:
df_raw.dtypes

NUM_BO                             int32
ANO_BO                             int16
ID_DELEGACIA                       int32
NOME_DEPARTAMENTO                 object
NOME_SECCIONAL                    object
DELEGACIA                         object
NOME_DEPARTAMENTO_CIRC            object
NOME_SECCIONAL_CIRC               object
NOME_DELEGACIA_CIRC               object
ANO                                int32
MES                                 int8
DATA_OCORRENCIA_BO        datetime64[ns]
HORA_OCORRENCIA_BO        datetime64[ns]
FLAG_STATUS                       object
RUBRICA                           object
DESDOBRAMENTO                     object
CONDUTA                           object
LATITUDE                          object
LONGITUDE                         object
CIDADE                            object
LOGRADOURO                        object
NUMERO_LOGRADOURO                 object
FLAG_STATUS.1                     object
DESCR_TIPO_PESSOA                 object
CONT_PESSOA     

In [16]:
np.shape(df_raw)

(801790, 30)

### Initial processing

In [17]:
train_cats(df_raw)
#df_raw.describe()

In [18]:
df_raw.dtypes

NUM_BO                             int32
ANO_BO                             int16
ID_DELEGACIA                       int32
NOME_DEPARTAMENTO               category
NOME_SECCIONAL                  category
DELEGACIA                       category
NOME_DEPARTAMENTO_CIRC          category
NOME_SECCIONAL_CIRC             category
NOME_DELEGACIA_CIRC             category
ANO                                int32
MES                                 int8
DATA_OCORRENCIA_BO        datetime64[ns]
HORA_OCORRENCIA_BO        datetime64[ns]
FLAG_STATUS                     category
RUBRICA                         category
DESDOBRAMENTO                   category
CONDUTA                         category
LATITUDE                        category
LONGITUDE                       category
CIDADE                          category
LOGRADOURO                      category
NUMERO_LOGRADOURO               category
FLAG_STATUS.1                   category
DESCR_TIPO_PESSOA               category
CONT_PESSOA     

We can specify the order to use for categorical variables if we wish:

In [19]:
display_all(df_raw.isnull().sum().sort_index()/len(df_raw))

ANO                       0.000000
ANO_BO                    0.000000
CIDADE                    0.000000
CONDUTA                   0.253066
CONT_PESSOA               0.000000
COR                       0.223560
DATA_OCORRENCIA_BO        0.004633
DELEGACIA                 0.000000
DESCR_GRAU_INSTRUCAO      0.453493
DESCR_PROFISSAO           0.455038
DESCR_TIPO_PESSOA         0.000085
DESDOBRAMENTO             0.960360
FLAG_STATUS               0.000000
FLAG_STATUS.1             0.000000
HORA_OCORRENCIA_BO        0.163906
IDADE_PESSOA              0.360222
ID_DELEGACIA              0.000000
LATITUDE                  0.967652
LOGRADOURO                0.000000
LONGITUDE                 0.967652
MES                       0.000000
NOME_DELEGACIA_CIRC       0.000000
NOME_DEPARTAMENTO         0.000000
NOME_DEPARTAMENTO_CIRC    0.000000
NOME_SECCIONAL            0.000000
NOME_SECCIONAL_CIRC       0.000000
NUMERO_LOGRADOURO         0.066564
NUM_BO                    0.000000
RUBRICA             

Nope! 
Luckily the UCI database has clean datasets.
We will look at dirty datasets in future classes.

But let's save this file for now, since it's already in format can we be stored and accessed efficiently.

In [20]:
os.makedirs(f'{PATH}tmp', exist_ok=True)
df_raw.to_feather(f'{PATH}tmp/crime-raw')

### Pre-processing

In the future we can simply read it from this fast format.

In [21]:
df_raw = pd.read_feather(f'{PATH}tmp/crime-raw')

We'll replace categories with their numeric codes, handle missing continuous values, and split the dependent variable into a separate variable.

In [22]:
df, y, _ = proc_df(df_raw, 'SEXO_PESSOA')

In [23]:
display_all(df_raw.describe(include='all').T)

,count,unique,top,freq,first,last,mean,std,min,25%,50%,75%,max
NUM_BO,801790,NaN,NaN,NaN,NaN,NaN,28923.2,117897,1,809,1759,3082,1.32769e+06
ANO_BO,801790,NaN,NaN,NaN,NaN,NaN,2009.01,0.144315,2009,2009,2009,2009,2017
ID_DELEGACIA,801790,NaN,NaN,NaN,NaN,NaN,99004.4,249542,10004,10336,20218,30218,900833
NOME_DEPARTAMENTO,801790,19,DECAP,464202,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NOME_SECCIONAL,801790,67,DEL.SEC.3º OESTE,74332,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
DELEGACIA,801790,469,DELEGACIA ELETRONICA,69804,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NOME_DEPARTAMENTO_CIRC,801790,2,DECAP,531941,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NOME_SECCIONAL_CIRC,801790,17,DEL.SEC.3º OESTE,89931,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
NOME_DELEGACIA_CIRC,801790,182,01º D.P. SE,17109,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ANO,801790,NaN,NaN,NaN,NaN,NaN,2009,0,2009,2009,2009,2009,2009


We now have something we can pass to a random forest!

In [25]:
m = RandomForestClassifier(n_jobs=-1)
m.fit(df, y)
m.score(df,y)

/home/sam/anaconda3/envs/fastai-cpu/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.9920852093440926